In [ ]:
# Zemberek-Python paketini yükleyin
# JPype'i yükleyin

!pip install jpype1
!pip install zemberek-python

In [ ]:
!pip install gensim==4.3.0


In [ ]:
!pip uninstall smart_open -y
!pip install smart_open


Found existing installation: smart-open 7.1.0
Uninstalling smart-open-7.1.0:
  Successfully uninstalled smart-open-7.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
# =====================================================
# 1) Gerekli Kütüphanelerin Import Edilmesi
# =====================================================
import pandas as pd
import re
import string
from zemberek import (
    TurkishSpellChecker, TurkishMorphology, TurkishTokenizer
)
from gensim.models import Word2Vec
from google.colab import files

# =====================================================
# 2) Zemberek Morfoloji ve Sınıfları Yükleme
# =====================================================
morphology = TurkishMorphology.create_with_defaults()
spell_checker = TurkishSpellChecker(morphology)
tokenizer = TurkishTokenizer.DEFAULT

# =====================================================
# 3) Veri Seti Okuma ve Ön İşleme
# =====================================================
excel_path = "/content/drive/MyDrive/Türkçe Nefret Söylemi Veriseti.xlsx"  # Colab'da veri seti yolu
# Excel dosyasını yüklediğinizden emin olun.
df = pd.read_excel(excel_path, sheet_name=None, header=1)
df = pd.concat(df.values(), ignore_index=True)

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"http\\S+", "", text)   # URL'leri kaldır
    text = re.sub(r"@\\w+", "", text)      # @kullanıcı kaldır
    text = re.sub(r"#\\w+", "", text)      # #hashtag kaldır
    text = re.sub(r"[0-9]", "", text)      # Sayıları kaldır
    text = re.sub(r"[^\w\s]", "", text)    # Noktalama işaretlerini kaldır
    return text.strip()

df['cleaned_text'] = df['Tweet'].apply(preprocess_text)




INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 10.34834337234497


2024-12-26 13:39:56,765 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 10.34834337234497



<ipython-input-4-efa923a7cb8f>:25: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel(excel_path, sheet_name=None, header=1)
<ipython-input-4-efa923a7cb8f>:25: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel(excel_path, sheet_name=None, header=1)
<ipython-input-4-efa923a7cb8f>:25: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel(excel_path, sheet_name=None, header=1)
<ipython-input-4-efa923a7cb8f>:25: FutureWarning: Inferring datetime64[ns] from data containing strings is de

In [ ]:

# =====================================================
# Optimum Yazım Düzeltme Fonksiyonu
# =====================================================
spell_cache = {}

def correct_spelling_optimized(text):
    corrected = []
    for word in text.split():
        if word in spell_cache:
            corrected.append(spell_cache[word])
        else:
            suggestions = spell_checker.suggest_for_word(word)
            corrected_word = suggestions[0] if suggestions else word
            spell_cache[word] = corrected_word
            corrected.append(corrected_word)
    return " ".join(corrected)

df['corrected_text'] = df['cleaned_text'].apply(correct_spelling_optimized)


In [ ]:
# =====================================================
# Tokenizasyon
# =====================================================
def tokenize_text(text):
    tokens = [str(token.content) for token in tokenizer.tokenize(text)]
    return tokens

df['tokenized'] = df['corrected_text'].apply(tokenize_text)

In [ ]:

# Lemmatization
# =====================================================
def lemmatize(tokens):
    lemmas = []
    for token in tokens:
        analysis = morphology.analyze_sentence(token)
        disambiguated = morphology.disambiguate(token, analysis)
        lemmas.extend([a.get_stem() for a in disambiguated.best_analysis()])
    return lemmas

df['lemmatized'] = df['tokenized'].apply(lemmatize)

In [ ]:

# =====================================================
# X ve Y (Lemmatized ve Etiket Kolonları) Kaydetme
# =====================================================
df[['lemmatized', 'Etiket']].to_csv("x_y_dataset.csv", index=False)

print("Lemmatized ve Etiket kolonları CSV olarak kaydedildi.")

# Dosyayı indir
files.download("x_y_dataset.csv")


Lemmatized ve Etiket kolonları CSV olarak kaydedildi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>